In [31]:
# Using Ripser

import numpy as np
from ripser import ripser
from persim import plot_diagrams
import operator
import timeit

## Persistence Landscapes: Algorithm 1

#### Define L as list

In [67]:
def PersistenceLanscape1(A):
    #L = np.array([])
    L = []
    k = 0

    # Sort A: read from right to left inside ()
    ind =  np.lexsort((-A[:,1], A[:,0])) 
    A = A[ind]

    while len(A) != 0:

        # pop first term
        bd, A = A[0], A[1:len(A)] 
        b, d = bd


        #L = np.extend(L, np.array([[-np.inf, 0], [b, 0], [(b+d)/2, (d-b)/2]]) ) # outer brackets for start of L_k
        L.append([ [-np.inf, 0], [b, 0], [(b+d)/2, (d-b)/2] ] ) # outer brackets for start of L_k

        while L[k][-1] != [np.inf, 0]:


            # Check if d is greater than all remaining pairs
            if (d  > A[:,1]).all(): # check dont need vector

                # add to end of L_k
                # L = np.append(L, np.array([ [d,0], [np.inf, 0] ]))
                L[k].extend([ [d,0], [np.inf, 0] ])

            else:
                # set (b', d')  to be the first term so that d' > d
                for i in range(len(A)):
                    if A[i][1] > d: 
                        # pop (b', d')

                        ind1 = [k for k in range(len(A) ) if k != i] 

                        bd_prime, A = A[i], A[ind1]

                        b_prime, d_prime = bd_prime
                        break


                # Case I    
                if b_prime > d:
                    #L = np.append(L, np.array([[d, 0] ]))
                    L[k].extend([ [d, 0] ])

                # Case II
                if b_prime >= d:
                    #L = np.append(L, np.array([ [b_prime, 0] ]))
                    L[k].extend([ [b_prime, 0] ])

                # Case III
                else:
                    #L = np.append(L, np.array([ [(b_prime + d)/2, (d-b_prime)/2] ]) ) 
                    L[k].extend([ [(b_prime + d)/2, (d-b_prime)/2] ])

                    # Push (b', d) into A in order
                    # find the first b_i in A so that b'<= b_i
                    for i in range(len(A)):
                        if b_prime <= A[i][0]:
                            ind2 = i # index to push (b', d) if b' != b_i 
                            break

                    # if b' = b_i 
                    # move index to the right one for every d_i so that d < d_i 
                    if b_prime == A[ind2][0]:
                        A_i = A[ A[:,0] == b_prime]

                        for j in range(len(A_i)):
                            if d < A_i[j][1]:
                                ind2 = ind2 + 1


                    A = np.insert(A, ind2 ,np.array([b_prime, d]), axis = 0)


                #L = np.append(L, np.array([ [(b_prime + d_prime)/2, (d_prime-b_prime)/2] ]) ) 
                L[k].extend([ [(b_prime + d_prime)/2, (d_prime-b_prime)/2] ])

                b,d = b_prime, d_prime # Set (b',d')= (b, d)     
        k += 1
    return(L)

#### Define L as array

In [103]:
def PersistenceLanscape2(A):
    L = np.array([])
    #L = []
    k = 0

    # Sort A: read from right to left inside ()
    ind =  np.lexsort((-A[:,1], A[:,0])) 
    A = A[ind]

    while len(A) != 0:

        # pop first term
        bd, A = A[0], A[1:len(A)] 
        b, d = bd

        # outer brackets for start of L_k
        L = np.insert(L, len(L), np.array([-np.inf, 0]) , axis = 0) 
        L = np.insert(L, len(L), np.array([b, 0]) , axis = 0) 
        L = np.insert(L, len(L), np.array([(b+d)/2, (d-b)/2]) , axis = 0) 
        #L.append([ [-np.inf, 0], [b, 0], [(b+d)/2, (d-b)/2] ] ) # outer brackets for start of L_k
        
        while (L[-1] != [np.inf, 0]).all():


            # Check if d is greater than all remaining pairs
            if (d  > A[:,1]).all(): # check dont need vector

                # add to end of L_k
                L = np.insert(L, len(L), np.array( [d,0] ), axis = 0)
                L = np.insert(L, len(L), np.array( [np.inf, 0] ), axis = 0)
                #L[k].extend([ [d,0], [np.inf, 0] ])

            else:
                # set (b', d')  to be the first term so that d' > d
                for i in range(len(A)):
                    if A[i][1] > d: 
                        # pop (b', d')

                        ind1 = [k for k in range(len(A) ) if k != i] 

                        bd_prime, A = A[i], A[ind1]

                        b_prime, d_prime = bd_prime
                        break


                # Case I    
                if b_prime > d:
                    L = np.insert(L, len(L), np.array([d, 0] ), axis = 0)
                    #L[k].extend([ [d, 0] ])

                # Case II
                if b_prime >= d:
                    L = np.insert(L, len(L), np.array( [b_prime, 0] ), axis = 0)
                    #L[k].extend([ [b_prime, 0] ])

                # Case III
                else:
                    L = np.insert(L, len(L), np.array([(b_prime + d)/2, (d-b_prime)/2]), axis = 0 ) 
                    #L[k].extend([ [(b_prime + d)/2, (d-b_prime)/2] ])

                    # Push (b', d) into A in order
                    # find the first b_i in A so that b'<= b_i
                    for i in range(len(A)):
                        if b_prime <= A[i][0]:
                            ind2 = i # index to push (b', d) if b' != b_i 
                            break

                    # if b' = b_i 
                    # move index to the right one for every d_i so that d < d_i 
                    if b_prime == A[ind2][0]:
                        A_i = A[ A[:,0] == b_prime]

                        for j in range(len(A_i)):
                            if d < A_i[j][1]:
                                ind2 = ind2 + 1


                    A = np.insert(A, ind2 ,np.array([b_prime, d]), axis = 0)


                L = np.insert(L, len(L), np.array([(b_prime + d_prime)/2, (d_prime-b_prime)/2] ), axis = 0 ) 
                #L[k].extend([ [(b_prime + d_prime)/2, (d_prime-b_prime)/2] ])

                b,d = b_prime, d_prime # Set (b',d')= (b, d)     
        k += 1
    return(L)

In [93]:
A = np.array([ [1.0, 5.0], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
PersistenceLanscape1(A)

[[[-inf, 0],
  [1.0, 0],
  [3.0, 2.0],
  [3.5, 1.5],
  [5.0, 3.0],
  [6.5, 1.5],
  [7.0, 2.0],
  [9.0, 0],
  [inf, 0]],
 [[-inf, 0], [2.0, 0], [3.5, 1.5], [5.0, 0], [6.5, 1.5], [8.0, 0], [inf, 0]],
 [[-inf, 0],
  [3.0, 0],
  [3.5, 0.5],
  [4.0, 0],
  [6.0, 0],
  [6.5, 0.5],
  [7.0, 0],
  [inf, 0]]]

In [104]:
A = np.array([ [1.0, 5.0], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
PersistenceLanscape2(A)

array([-inf,  0. ,  1. ,  0. ,  3. ,  2. ,  3.5,  1.5,  5. ,  3. ,  6.5,
        1.5,  7. ,  2. ,  9. ,  0. ,  inf,  0. , -inf,  0. ,  2. ,  0. ,
        3.5,  1.5,  5. ,  0. ,  6.5,  1.5,  8. ,  0. ,  inf,  0. , -inf,
        0. ,  3. ,  0. ,  3.5,  0.5,  4. ,  0. ,  6. ,  0. ,  6.5,  0.5,
        7. ,  0. ,  inf,  0. ])

## Using Timeit to compare array vs. list

In [58]:
SETUP_CODE1 = ''' 
from __main__ import PersistenceLanscape1
import numpy as np'''

TEST_CODE1 = ''' 
A = np.array([ [1.0, 5.0], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
PersistenceLanscape1(A) 
    '''

times1 = timeit.repeat(setup = SETUP_CODE1, 
                          stmt = TEST_CODE1,  
                      repeat = 10,
                      number = 10000) # time it takes to execute the snippet 10,000 times

In [106]:
SETUP_CODE2 = ''' 
from __main__ import PersistenceLanscape2
import numpy as np'''

TEST_CODE2 = ''' 
A = np.array([ [1.0, 5.0], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
PersistenceLanscape2(A) 
    '''

times2 = timeit.repeat(setup = SETUP_CODE2, 
                          stmt = TEST_CODE2,  
                      repeat = 10,
                      number = 10000) # time it takes to execute the snippet 10,000 times

In [64]:
times_list = np.array([])
times_list = np.append(times_list, times1)
times_list_avg = np.mean(np.append(times_list, times1))
times_list

array([1.62714347, 1.61685451, 1.59747162, 1.59776129, 1.62404009,
       1.60066211, 1.61193538, 1.58486053, 1.60761017, 1.63683735])

In [109]:
times_array = np.array([])
times_array = np.append(times_list, times2)
times_array_avg = np.mean(np.append(times_list, times2))
times_array_avg

4.438789762650027

In [110]:
print('time_list average is ', times_list_avg,'time_array average is ', times_array_avg)

time_list average is  1.610517653299894 time_array average is  4.438789762650027


### Algorithm with print statements 

In [21]:
A = np.array([ [1.0, 5.0], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
print("A is", A)
#L = np.array([])
L = []
k = 0

# Sort A: read from right to left inside ()
ind =  np.lexsort((-A[:,1], A[:,0])) 
A = A[ind]

while len(A) != 0:
    print('outer loop')
    print("k is",k)
    
    # pop first term
    bd, A = A[0], A[1:len(A)] 
    b, d = bd
    
    
    #L = np.extend(L, np.array([[-np.inf, 0], [b, 0], [(b+d)/2, (d-b)/2]]) ) # outer brackets for start of L_k
    L.append([ [-np.inf, 0], [b, 0], [(b+d)/2, (d-b)/2] ] ) # outer brackets for start of L_k
    
    print("L is",L)
    while L[k][-1] != [np.inf, 0]:
        print("(b,d) is", [b, d])
        print('inner loop')

        # Check if d is greater than all remaining pairs
        if (d  > A[:,1]).all(): # check dont need vector
            
            # add to end of L_k
            # L = np.append(L, np.array([ [d,0], [np.inf, 0] ]))
            L[k].extend([ [d,0], [np.inf, 0] ])

        else:
            # set (b', d')  to be the first term so that d' > d
            for i in range(len(A)):
                if A[i][1] > d: 
                    # pop (b', d')
                    
                    ind1 = [k for k in range(len(A) ) if k != i] 
                    
                    bd_prime, A = A[i], A[ind1]
                    
                    b_prime, d_prime = bd_prime
                    break
            print("(b',d') is", [b_prime, d_prime])

            # Case I    
            if b_prime > d:
                #L = np.append(L, np.array([[d, 0] ]))
                L[k].extend([ [d, 0] ])

            # Case II
            if b_prime >= d:
                #L = np.append(L, np.array([ [b_prime, 0] ]))
                L[k].extend([ [b_prime, 0] ])

            # Case III
            else:
                #L = np.append(L, np.array([ [(b_prime + d)/2, (d-b_prime)/2] ]) ) 
                L[k].extend([ [(b_prime + d)/2, (d-b_prime)/2] ])
                
                # Push (b', d) into A in order
                # find the first b_i in A so that b'<= b_i
                for i in range(len(A)):
                    if b_prime <= A[i][0]:
                        ind2 = i # index to push (b', d) if b' != b_i 
                        break
                
                # if b' = b_i 
                # move index to the right one for every d_i so that d < d_i 
                if b_prime == A[ind2][0]:
                    A_i = A[ A[:,0] == b_prime]
                    
                    for j in range(len(A_i)):
                        if d < A_i[j][1]:
                            ind2 = ind2 + 1
                    print("(b',d) is", [b_prime, d])
                            
                A = np.insert(A, ind2 ,np.array([b_prime, d]), axis = 0)


            #L = np.append(L, np.array([ [(b_prime + d_prime)/2, (d_prime-b_prime)/2] ]) ) 
            L[k].extend([ [(b_prime + d_prime)/2, (d_prime-b_prime)/2] ])
            
            b,d = b_prime, d_prime # Set (b',d')= (b, d)     
            print("A is", A)
    k += 1
    

A is [[1. 5.]
 [2. 8.]
 [3. 4.]
 [5. 9.]
 [6. 7.]]
outer loop
k is 0
L is [[[-inf, 0], [1.0, 0], [3.0, 2.0]]]
(b,d) is [1.0, 5.0]
inner loop
(b',d') is [2.0, 8.0]
A is [[2. 5.]
 [3. 4.]
 [5. 9.]
 [6. 7.]]
(b,d) is [2.0, 8.0]
inner loop
(b',d') is [5.0, 9.0]
A is [[2. 5.]
 [3. 4.]
 [5. 8.]
 [6. 7.]]
(b,d) is [5.0, 9.0]
inner loop
outer loop
k is 1
L is [[[-inf, 0], [1.0, 0], [3.0, 2.0], [3.5, 1.5], [5.0, 3.0], [6.5, 1.5], [7.0, 2.0], [9.0, 0], [inf, 0]], [[-inf, 0], [2.0, 0], [3.5, 1.5]]]
(b,d) is [2.0, 5.0]
inner loop
(b',d') is [5.0, 8.0]
A is [[3. 4.]
 [6. 7.]]
(b,d) is [5.0, 8.0]
inner loop
outer loop
k is 2
L is [[[-inf, 0], [1.0, 0], [3.0, 2.0], [3.5, 1.5], [5.0, 3.0], [6.5, 1.5], [7.0, 2.0], [9.0, 0], [inf, 0]], [[-inf, 0], [2.0, 0], [3.5, 1.5], [5.0, 0], [6.5, 1.5], [8.0, 0], [inf, 0]], [[-inf, 0], [3.0, 0], [3.5, 0.5]]]
(b,d) is [3.0, 4.0]
inner loop
(b',d') is [6.0, 7.0]
A is []
(b,d) is [6.0, 7.0]
inner loop


### Sorting using .sort

In [2]:
test1 =[  (2.0, 8.0), (4.0, 3.0), (5.0, 9.0), (6.0,9.0), (6.0, 7.0),(1.0, 5.0) ]
dtype =[('first_coord', float), ('second_coord', float)]
a = np.array(test1, dtype=dtype) 
a

array([(2., 8.), (4., 3.), (5., 9.), (6., 9.), (6., 7.), (1., 5.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

Sort based on first coordinate, second coordinate sorted automatically increasing

In [4]:
sort1 = np.sort(a, order='first_coord')
sort1

array([(1., 5.), (2., 8.), (4., 3.), (5., 9.), (6., 7.), (6., 9.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

Sorts both coordinates backwards

In [24]:
a[:: -1].sort(order = 'second_coord')
a

array([(6., 9.), (5., 9.), (2., 8.), (6., 7.), (1., 5.), (4., 3.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

In [21]:
np.sort(a, order='first_coord')

array([(1., 5.), (2., 8.), (4., 3.), (5., 9.), (6., 7.), (6., 9.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

In [12]:
temp = np.random.randint(1,10, 10)  
temp

array([4, 4, 1, 3, 4, 3, 7, 2, 3, 6])

In [14]:
temp[:: -1].sort()
temp

array([7, 6, 4, 4, 4, 3, 3, 3, 2, 1])

### Sorting using lexsort

In [48]:
a = np.array([[3, 2], [6, 2], [3, 6], [3, 4], [5, 3]])

ind1 =  np.lexsort((a[:,1], a[:,0])) 
ind2 =  np.lexsort((-a[:,1], a[:,0])) 
a

array([[3, 2],
       [6, 2],
       [3, 6],
       [3, 4],
       [5, 3]])

In [49]:
a[ind1]

array([[3, 2],
       [3, 4],
       [3, 6],
       [5, 3],
       [6, 2]])

In [50]:
a[ind2]

array([[3, 6],
       [3, 4],
       [3, 2],
       [5, 3],
       [6, 2]])

In [51]:
ind

array([2, 3, 0, 4, 1])